## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [3]:
import torch
import numpy as np

# Training data

In [8]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [9]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [10]:
# Convert np to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs, inputs.shape)
print(targets, targets.shape)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]) torch.Size([5, 3])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]) torch.Size([5, 2])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [11]:
# weights and biases
# Initialize with random value
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.7489,  0.1692, -1.2356],
        [ 0.6721,  1.5591,  1.9033]], requires_grad=True)
tensor([2.0340, 0.2841], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [12]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [13]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 87.9040, 235.6519],
        [ 96.9880, 320.4610],
        [105.1894, 378.0699],
        [141.9740, 206.3062],
        [ 52.4528, 329.5664]], grad_fn=<AddBackward0>)


In [14]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [15]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [16]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(22707.8730, grad_fn=<DivBackward0>)


The lower of the loss, the better of the model.

# Compute gradients

In [17]:
# Compute gradients
loss.backward()

In [18]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 1.7489,  0.1692, -1.2356],
        [ 0.6721,  1.5591,  1.9033]], requires_grad=True)
tensor([[ 2266.3931,   400.0468,   498.9639],
        [17036.5879, 18149.0312, 11277.3145]])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [19]:
w
w.grad

tensor([[ 2266.3931,   400.0468,   498.9639],
        [17036.5879, 18149.0312, 11277.3145]])

In [20]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the `learning rate` of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [21]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(22707.8730, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [22]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [23]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 85.7667, 206.2040],
        [ 94.2540, 281.7670],
        [102.3919, 332.3855],
        [139.3055, 176.9501],
        [ 50.1555, 292.4920]], grad_fn=<AddBackward0>)


In [24]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(15855.9043, grad_fn=<DivBackward0>)


In [25]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 2050.6125,   181.2592,   361.2277],
        [13997.0195, 14884.7832,  9262.5635]])
tensor([ 18.1747, 165.9597])


In [26]:
# Adjust the weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [27]:
# New weights and biases
print(w)
print(b)

tensor([[ 1.7057,  0.1634, -1.2442],
        [ 0.3618,  1.2287,  1.6979]], requires_grad=True)
tensor([2.0336, 0.2804], requires_grad=True)


In [28]:
# With new w and b, the model should have lower loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(11232.1299, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [33]:
# Train for 100 epochs
for i in range(300):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [34]:
# Let's see the new loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(62.3254, grad_fn=<DivBackward0>)


In [35]:
# Prediction
preds

tensor([[ 58.7949,  70.1885],
        [ 74.6568, 102.2855],
        [133.2506, 129.4795],
        [ 27.7143,  36.2220],
        [ 85.3143, 122.4643]], grad_fn=<AddBackward0>)

In [36]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now queit close to the target variables.